In [23]:
#!pip install psycopg2

In [39]:
import psycopg2
from psycopg2 import Error

try:
    # Connect to an existing database
    connection = psycopg2.connect(user="postgres",host="localhost", port="5432", database="etlcode",password="")
except Exception as e:
    print(e)
    

fe_sendauth: no password supplied



In [40]:
#CREATE EXTENSION IF NOT EXISTS "uuid-ossp";
# SELECT uuid_generate_v4();

In [41]:
create_dept_table = """CREATE TABLE IF NOT EXISTS department(
id UUID PRIMARY KEY DEFAULT uuid_generate_v4(),
name TEXT,
salary_increment NUMERIC,
UNIQUE(name)
);
"""

In [42]:
create_emp_table = """
CREATE TABLE IF NOT EXISTS employee(
    id UUID PRIMARY KEY DEFAULT uuid_generate_v4(),
    first_name TEXT,
    last_name TEXT,
    salary NUMERIC,
    department_id uuid,
    UNIQUE(first_name,last_name),
    CONSTRAINT fk_department
    FOREIGN KEY(department_id) 
    REFERENCES department(id)
);
"""

In [43]:
create_up_sal_table = """
CREATE TABLE IF NOT EXISTS updated_salaries(
employee_id uuid,
updated_salary NUMERIC,
UNIQUE(employee_id),
CONSTRAINT fk_employee
    FOREIGN KEY(employee_id) 
    REFERENCES employee(id)
)
"""

In [58]:
cur = connection.cursor()
cur.execute(create_dept_table)
cur.execute(create_emp_table)
cur.execute(create_up_sal_table)
cur.close()
connection.commit()

In [51]:
cur = connection.cursor()
cur.execute("DROP TABLE updated_salaries")
cur.execute("DROP TABLE employee")
cur.execute("DROP TABLE department")



In [53]:
cur.close()
connection.commit()


In [143]:
!pipenv --packages org.postgresql:postgresql:42.2.10

Usage: pipenv [OPTIONS] COMMAND [ARGS]...
Try "pipenv -h" for help.

Error: no such option: --packages


In [ ]:
#search for it 
"""
--MERGE SQL statement - Part 2

--Synchronize the target table with refreshed data from source table
MERGE Products AS TARGET
USING UpdatedProducts AS SOURCE 
ON (TARGET.ProductID = SOURCE.ProductID) 
--When records are matched, update the records if there is any change
WHEN MATCHED AND TARGET.ProductName <> SOURCE.ProductName OR TARGET.Rate <> SOURCE.Rate 
THEN UPDATE SET TARGET.ProductName = SOURCE.ProductName, TARGET.Rate = SOURCE.Rate 
--When no records are matched, insert the incoming records from source table to target table
WHEN NOT MATCHED BY TARGET 
THEN INSERT (ProductID, ProductName, Rate) VALUES (SOURCE.ProductID, SOURCE.ProductName, SOURCE.Rate)
--When there is a row that exists in target and same record does not exist in source then delete this record target
WHEN NOT MATCHED BY SOURCE 
THEN DELETE 
--$action specifies a column of type nvarchar(10) in the OUTPUT clause that returns 
--one of three values for each row: 'INSERT', 'UPDATE', or 'DELETE' according to the action that was performed on that row
OUTPUT $action, 
DELETED.ProductID AS TargetProductID, 
DELETED.ProductName AS TargetProductName, 
DELETED.Rate AS TargetRate, 
INSERTED.ProductID AS SourceProductID, 
INSERTED.ProductName AS SourceProductName, 
INSERTED.Rate AS SourceRate; 

SELECT @@ROWCOUNT;
GO

"""